In [1]:
# Import basic liabraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [4]:
!pip install scikit-learn xgboost catboost mlxtend


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/03/e6/4aef6799badc2693548559bad5b56d56cfe89eada337c815fdfe92175250/xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl.metadata
  Using cached xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl.metadata (2.0 kB)
  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/03/9f/5da788602cb9a2ce70abd9b3f9650dc53e6ec834e746b8dd053cb4314ca3/catboost-1.2.5-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Using cached catboost-1.2.5-cp311-cp311-macosx_11_0_universal2.whl.metadata (1.2 kB)
  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/1c/07/512f6a780239ad6ce06ce2aa7b4067583f5ddcfc7703a964a082c706a070/mlxtend-0.23.1-py3-none-any.whl.metadata
  Using cached mlxtend-0.23.1-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b63

In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score


In [6]:
# Load the data
X_train = pd.read_csv('X_train_wFE.csv').drop(columns='ID', axis=1)
y_train = pd.read_csv('Y_train.csv')
X_test_initial = pd.read_csv('X_test_wFE.csv')
test_ids = X_test_initial['ID']
X_test = X_test_initial.drop(columns='ID', axis=1)

In [7]:
# Base models
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gbm', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('catboost', CatBoostClassifier(verbose=0, random_state=42))
]

# Stacking classifier with LogisticRegression as meta-learner
stack = StackingClassifier(
    estimators=estimators, 
    final_estimator=LogisticRegression(),
    cv=5
)

# Train the stacked model
stack.fit(X_train, y_train)

# Predict the test set
y_pred = stack.predict(X_test) 
# Ignore all the validation part since it can be done quickly by submitting the results to the leaderboard


In [9]:
# Combine IDs with predictions for a submission
results_df = pd.DataFrame({
    'ID': test_ids,
    'Overall_Experience': y_pred  # Adjust based on your model output structure
})

# Save or return results
results_df.to_csv('submisson5.csv', index=False)

In [14]:
results_df.shape

(35602, 2)

## Hyper-parameter tuning

In [10]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Hyperparameters grid for each model
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 8]
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.15],
    'subsample': [0.8, 1.0]
}


param_grid_catboost = {
    'iterations': [100, 200],
    'learning_rate': [0.05, 0.1, 0.15],
    'depth': [4, 6, 8]
}

param_grid_meta = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}


In [11]:
from sklearn.model_selection import RandomizedSearchCV

def perform_random_search(model, params, X_train, y_train, n_iter=50, cv=3):
    """ Helper function to perform randomized search """
    rs = RandomizedSearchCV(model, params, n_iter=n_iter, cv=cv, verbose=1, random_state=42, n_jobs=-1)
    rs.fit(X_train, y_train)
    print(f"Best parameters for {model.__class__.__name__}: {rs.best_params_}")
    return rs.best_estimator_

# Perform randomized search for each classifier
best_rf = perform_random_search(RandomForestClassifier(random_state=42), param_grid_rf, X_train, y_train)
best_gb = perform_random_search(GradientBoostingClassifier(random_state=42), param_grid_gb, X_train, y_train)
best_xgb = perform_random_search(XGBClassifier(random_state=42), param_grid_xgb, X_train, y_train)
best_cat = perform_random_search(CatBoostClassifier(random_state=42, verbose=0), param_grid_catboost, X_train, y_train)
best_meta = perform_random_search(LogisticRegression(random_state=42), param_grid_meta, X_train, y_train)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

Best parameters for RandomForestClassifier: {'n_estimators': 300, 'min_samples_leaf': 1, 'max_depth': 30}
Fitting 3 folds for each of 27 candidates, totalling 81 fits


...

/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

Best parameters for GradientBoostingClassifier: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1}
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for XGBClassifier: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.1}
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best parameters for CatBoostClassifier: {'learning_rate': 0.15, 'iterations': 200, 'depth': 8}
Fitting 3 folds for each of 6 candidates, totalling 18 fits


/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/eyvone/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Best parameters for LogisticRegression: {'solver': 'liblinear', 'C': 1}


In [12]:
# Setup the stacking ensemble with tuned models
stack = StackingClassifier(
    estimators=[
        ('rf', best_rf),
        ('gb', best_gb),
        ('xgb', best_xgb),
        ('catboost', best_cat)
    ],
    final_estimator=best_meta,
    cv=5
)

# Train the stacking ensemble
stack.fit(X_train, y_train)

# Evaluate the final ensemble
y_pred_tuned = stack.predict(X_test)



In [13]:
# Combine IDs with predictions for a submission
results_df_tuned = pd.DataFrame({
    'ID': test_ids,
    'Overall_Experience': y_pred_tuned  # Adjust based on your model output structure
})

# Save or return results
results_df_tuned.to_csv('submisson6.csv', index=False)

## the tuning does not improve the result 